In [21]:
"""
PREDICTION DEMO – JUPYTER NOTEBOOK
==================================
Loads the champion classification model and evaluates it on a
small, stratified set of hold-out reviews.

The notebook is organised in clear, numbered sections:
    1) File checks
    2) Load model + artefacts
    3) Prepare test data
    4) Select a stratified sample
    5) Run predictions
    6) Performance metrics
    7) Per-review diagnostics
"""

# ------------------------------------------------------------------
# Standard libraries
# ------------------------------------------------------------------
import pickle
import random
import textwrap
from pathlib import Path
import warnings

# ------------------------------------------------------------------
# Third-party libraries
# ------------------------------------------------------------------
import pandas as pd
import numpy as np
from sklearn.metrics import (
    accuracy_score, classification_report,
    mean_absolute_error, confusion_matrix
)

warnings.filterwarnings("ignore", category=UserWarning)

print("🚀  PREDICTION DEMO – champion model")
print("=" * 60)


🚀  PREDICTION DEMO – champion model


In [20]:
# =============================================================================
# 1) CHECK REQUIRED FILES & PATHS
# =============================================================================
from pathlib import Path

MODEL_PKL   = Path("../src/models/best_classification_model.pkl")
FEAT_PKL    = Path("../src/models/feature_info.pkl")
SPLIT_PKL   = Path("../src/models/train_test_splits.pkl")
DEFAULT_CSV = Path("../src/data/processed/temu_reviews_preprocessed.csv")

print("📂 File availability:")
file_flags = {
    "Model bundle"       : MODEL_PKL.exists(),
    "Feature metadata"   : FEAT_PKL.exists(),
    "Train/Test splits"  : SPLIT_PKL.exists(),
    "Pre-processed CSV"  : DEFAULT_CSV.exists()
}

for label, ok in file_flags.items():
    mark = "✅" if ok else "❌"
    print(f"   {mark} {label}")

missing_core = [name for name, ok in file_flags.items()
                if not ok and name in {"Model bundle", "Feature metadata"}]

if missing_core:
    msg = ", ".join(missing_core)
    raise FileNotFoundError(
        f"Critical file(s) missing: {msg}. "
        "Please run the training notebook first."
    )


📂 File availability:
   ✅ Model bundle
   ✅ Feature metadata
   ✅ Train/Test splits
   ✅ Pre-processed CSV


In [14]:
# =============================================================================
# 2) LOAD MODEL AND SUPPORT FILES
# =============================================================================
import pickle
from pathlib import Path
from sklearn.preprocessing import LabelEncoder

MODEL_PKL = Path("../src/models/best_classification_model.pkl")
FEAT_PKL  = Path("../src/models/feature_info.pkl")

print("\n📦 Loading model artefacts …")

# ------------------------------------------------------------------
# 1) Load the champion model bundle
# ------------------------------------------------------------------
try:
    with MODEL_PKL.open("rb") as fp:
        mdl_dict = pickle.load(fp)

    clf        = mdl_dict["model"]          # fitted estimator
    model_name = mdl_dict["model_name"]
    test_acc   = mdl_dict.get("test_accuracy", "N/A")
    w_f1       = mdl_dict.get("weighted_f1", "N/A")

    print(f"✅ Model loaded : {model_name}")
    if w_f1 != "N/A":
        print(f"   Weighted F1  : {w_f1:.3f}")
    if test_acc != "N/A":
        print(f"   Test accuracy: {test_acc:.3f}")

except Exception as e:
    raise RuntimeError(f"❌ Could not load model pickle: {e}")

# ------------------------------------------------------------------
# 2) Load vectoriser & metadata
# ------------------------------------------------------------------
try:
    with FEAT_PKL.open("rb") as fp:
        feat = pickle.load(fp)

    vectorizer         = feat["tfidf_vectorizer"]
    feature_count      = feat.get("feature_count",
                                  len(vectorizer.get_feature_names_out()))
    numerical_features = feat.get("numerical_features", [])
    class_weights      = feat.get("class_weights", {})
    label_encoder      = feat.get("label_encoder")  # may be None

    print(f"✅ Vectoriser loaded : {feature_count:,} features")
    print(f"✅ Numerical columns : {numerical_features}")
    if class_weights:
        print(f"✅ Class weights     : {class_weights}")
    if label_encoder is None:
        # fall-back: identity encoder 1★…5★ → 0…4
        label_encoder = LabelEncoder().fit([1, 2, 3, 4, 5])
        print("🔧 Label encoder created (identity 1-5 → 0-4)")
    else:
        print("✅ Label encoder loaded from feature_info")

except Exception as e:
    raise RuntimeError(f"❌ Could not load feature artefacts: {e}")


📦 Loading model artefacts …
✅ Model loaded : Stacking
   Weighted F1  : 0.725
   Test accuracy: 0.733
✅ Vectoriser loaded : 5,000 features
✅ Numerical columns : ['word_count', 'char_count', 'sentence_count', 'avg_word_length', 'exclamation_count', 'question_count', 'capital_ratio', 'sentiment_compound', 'sentiment_pos', 'sentiment_neu', 'sentiment_neg']
✅ Class weights     : {np.int64(1): np.float64(0.38278940027894004), np.int64(2): np.float64(3.1510907003444317), np.int64(3): np.float64(4.235493827160494), np.int64(4): np.float64(2.463734290843806), np.int64(5): np.float64(0.7001530612244898)}
✅ Label encoder loaded from feature_info


In [15]:
# =============================================================================
# 3) PREPARE TEST DATA
# =============================================================================
import pickle, pandas as pd
from pathlib import Path

print("\n📊 Preparing test data …")

# ------------------------------------------------------------------
# paths
# ------------------------------------------------------------------
SPLIT_PKL   = Path("../src/models/train_test_splits.pkl")
DEFAULT_CSV = Path("../data/preprocessed_reviews.csv")

# make sure numerical_features exists
if "numerical_features" not in globals():
    numerical_features = []      # will be updated later if CSV contains them

# decide which source to use
use_splits = SPLIT_PKL.exists()
use_csv    = DEFAULT_CSV.exists() and not use_splits

# ------------------------------------------------------------------
# 1) load saved train/test splits
# ------------------------------------------------------------------
if use_splits:
    print("📋 Using stored train/test splits …")
    try:
        with SPLIT_PKL.open("rb") as fp:
            splits = pickle.load(fp)

        X_test  = splits["combined"]["X_test"]
        y_test  = splits["combined"]["y_test"]
        print(f"✅ Test split loaded : {X_test.shape[0]:,} samples "
              f"({X_test.shape[1]:,} features)")
        print(f"   Rating distribution: {dict(pd.Series(y_test).value_counts().sort_index())}")
        data_source = "splits"

    except Exception as e:
        print(f"❌ Could not load splits: {e}")
        use_splits = False
        use_csv    = DEFAULT_CSV.exists()

# ------------------------------------------------------------------
# 2) fallback: load the pre-processed CSV
# ------------------------------------------------------------------
if use_csv and not use_splits:
    print("📋 Using pre-processed CSV …")
    try:
        df = pd.read_csv(DEFAULT_CSV)
        df = df.dropna(subset=["processed_text", "ReviewRating"])
        df["ReviewRating"] = df["ReviewRating"].astype(int)
        df = df[df["ReviewRating"].between(1, 5)]        # keep ratings 1-5

        print(f"✅ CSV loaded : {len(df):,} valid reviews")
        print(f"   Rating distribution: {dict(df['ReviewRating'].value_counts().sort_index())}")

        # check which numeric columns are present
        avail_num   = [c for c in numerical_features if c in df.columns]
        missing_num = [c for c in numerical_features if c not in df.columns]
        if missing_num:
            print(f"⚠️ Missing numeric features: {missing_num}")
            print(f"✅ Available numeric features: {avail_num}")
            numerical_features = avail_num      # update list

        data_source = "csv"

    except Exception as e:
        raise RuntimeError(f"❌ Could not load CSV: {e}")

# ------------------------------------------------------------------
# 3) no data found
# ------------------------------------------------------------------
if not use_splits and not use_csv:
    raise FileNotFoundError("No test data available: neither split pickle nor CSV found.")


📊 Preparing test data …
📋 Using stored train/test splits …
✅ Test split loaded : 2,745 samples (5,011 features)
   Rating distribution: {1: np.int64(1434), 2: np.int64(174), 3: np.int64(130), 4: np.int64(223), 5: np.int64(784)}


In [16]:
# =============================================================================
# 4) SELECT A STRATIFIED SAMPLE
# =============================================================================
import numpy as np, random
from scipy.sparse import hstack
from sklearn.preprocessing import StandardScaler

print("\n🎲 Selecting stratified test samples …")

N_SAMPLES = 15          # total reviews to inspect
rng       = np.random.default_rng(42)

# -----------------------------------------------------------
# Helper to pick k indices per star rating
# -----------------------------------------------------------
def stratified_indices(labels, k_total):
    """Return <=k_total indices, equally split across present ratings."""
    unique = np.unique(labels)
    k_per  = max(1, k_total // len(unique))
    idx = []
    for star in unique:
        star_idx = np.where(labels == star)[0].tolist()
        take = min(k_per, len(star_idx))
        idx.extend(rng.choice(star_idx, size=take, replace=False))
    # top-up if we’re short
    if len(idx) < k_total:
        remaining = [i for i in range(len(labels)) if i not in idx]
        extra = rng.choice(remaining, size=k_total-len(idx), replace=False)
        idx.extend(extra)
    return idx[:k_total]

# -----------------------------------------------------------
# A) use stored splits
# -----------------------------------------------------------
if data_source == "splits":
    labels = np.array(y_test)
    idx    = stratified_indices(labels, N_SAMPLES)

    X_sample  = X_test[idx]
    y_true    = labels[idx]
    sample_texts = None   # raw text is not part of the split object

# -----------------------------------------------------------
# B) use CSV   (df already loaded earlier)
# -----------------------------------------------------------
elif data_source == "csv":
    idx_df = stratified_indices(df["ReviewRating"].values, N_SAMPLES)
    samples_df = df.iloc[idx_df].reset_index(drop=True)

    # Which feature combo did the model expect?
    combo = mdl_dict.get("feature_combination", "combined")
    print(f"🔧 Model expects feature set: {combo}")

    # ---- TF-IDF block
    X_tfidf = vectorizer.transform(samples_df["processed_text"])

    # ---- numerical block (if any)
    if numerical_features:
        X_num = samples_df[numerical_features].values
        if "scaler" in feature_info:
            scaler = feature_info["scaler"]        # trained scaler
            X_num_scaled = scaler.transform(X_num)
        else:                                      # fallback (not ideal)
            print("⚠️  Using fresh StandardScaler – may distort results")
            X_num_scaled = StandardScaler().fit_transform(X_num)
    else:
        X_num_scaled = None

    # ---- build X_sample according to combo
    if combo == "tfidf_only":
        X_sample = X_tfidf
    elif combo == "numerical_only":
        if X_num_scaled is None:
            raise ValueError("Model requires numeric features but none are present.")
        X_sample = X_num_scaled
    else:   # combined
        if X_num_scaled is None:
            print("⚠️ Model expects combined features, but numeric part is missing. Falling back to TF-IDF only.")
            X_sample = X_tfidf
        else:
            X_sample = hstack([X_tfidf, X_num_scaled])

    y_true       = samples_df["ReviewRating"].values
    sample_texts = samples_df["processed_text"].values

# -----------------------------------------------------------
# Summary
# -----------------------------------------------------------
print(f"✅ Sample size: {len(y_true)}")
print(f"   Rating distribution: {dict(pd.Series(y_true).value_counts().sort_index())}")


🎲 Selecting stratified test samples …
✅ Sample size: 15
   Rating distribution: {1: np.int64(3), 2: np.int64(3), 3: np.int64(3), 4: np.int64(3), 5: np.int64(3)}


In [17]:
# =============================================================================
# 5) MAKE PREDICTIONS
# =============================================================================
import numpy as np

print("\n🔮 Generating predictions …")

try:
    # -----------------------------------------------------------
    # 1) Raw model output (encoded labels)
    # -----------------------------------------------------------
    y_pred_enc = clf.predict(X_sample)
    print(f"🔧 Raw model output (unique): {sorted(set(y_pred_enc))}")

    # -----------------------------------------------------------
    # 2) Detect the label scheme and map to 1-5 stars
    # -----------------------------------------------------------
    if np.min(y_pred_enc) >= 1 and np.max(y_pred_enc) <= 5:
        # model is already on 1-5 scale
        y_pred = y_pred_enc
        print("✅ Model uses 1–5 encoding")
    elif np.min(y_pred_enc) >= 0 and np.max(y_pred_enc) <= 4:
        # model is on 0-4 → shift
        y_pred = y_pred_enc + 1
        print("✅ Model uses 0–4 encoding → shifted to 1–5")
    else:
        # unexpected range – clip as last resort
        print(f"⚠️ Unexpected labels {sorted(set(y_pred_enc))} – clipping to 1–5")
        y_pred = np.clip(y_pred_enc, 1, 5).astype(int)

    # final sanity-check
    assert y_pred.min() >= 1 and y_pred.max() <= 5, "label mapping failed"

    # -----------------------------------------------------------
    # 3) Confidence scores if available
    # -----------------------------------------------------------
    if hasattr(clf, "predict_proba"):
        y_proba    = clf.predict_proba(X_sample)
        confidence = y_proba.max(axis=1)
        print(f"✅ Mean confidence: {confidence.mean():.1%}")

        buckets = {
            "<50%" : confidence < 0.50,
            "50–70%" : (confidence >= .50) & (confidence < .70),
            "70–85%" : (confidence >= .70) & (confidence < .85),
            "85–95%" : (confidence >= .85) & (confidence < .95),
            ">95%"   : confidence >= .95,
        }
        print("📊 Confidence distribution:")
        for label, mask in buckets.items():
            count = int(mask.sum())
            if count:
                print(f"   {label:<7}: {count} sample(s)")
    else:
        confidence = None
        print("ℹ️  Model exposes no predict_proba – confidence skipped")

    # -----------------------------------------------------------
    # 4) Quick preview
    # -----------------------------------------------------------
    print("\n🔍 First five predictions:")
    for i in range(min(5, len(y_pred))):
        conf_str = f" (conf: {confidence[i]:.1%})" if confidence is not None else ""
        print(f"   Sample {i+1}: true {y_true[i]}★ → pred {y_pred[i]}★{conf_str}")

    print("✅ Predictions generated successfully")

except Exception as e:
    print(f"❌ Prediction error: {e}")
    print("🔎 Debug info:")
    print(f"   X_sample shape : {getattr(X_sample,'shape',None)}")
    print(f"   Estimator type : {type(clf)}")
    print(f"   Model name     : {model_name}")
    raise


🔮 Generating predictions …
🔧 Raw model output (unique): [np.int64(3)]
✅ Model uses 1–5 encoding
✅ Mean confidence: 32.5%
📊 Confidence distribution:
   <50%   : 15 sample(s)

🔍 First five predictions:
   Sample 1: true 1★ → pred 3★ (conf: 31.4%)
   Sample 2: true 1★ → pred 3★ (conf: 35.2%)
   Sample 3: true 1★ → pred 3★ (conf: 32.0%)
   Sample 4: true 2★ → pred 3★ (conf: 33.6%)
   Sample 5: true 2★ → pred 3★ (conf: 35.4%)
✅ Predictions generated successfully


In [18]:
# =============================================================================
# 6) PERFORMANCE METRICS
# =============================================================================

from sklearn.metrics import (
    accuracy_score, mean_absolute_error,
    classification_report, confusion_matrix
)
import numpy as np
import pandas as pd

print("\n📈  PERFORMANCE ANALYSIS")
print("=" * 60)

# -----------------------------------------------------------
# 1) Quick sanity check
# -----------------------------------------------------------
print("🔍 Debug:")
print(f"   y_true unique : {sorted(set(y_true))}")
print(f"   y_pred unique : {sorted(set(y_pred))}")

# -----------------------------------------------------------
# 2) Basic metrics
# -----------------------------------------------------------
acc  = accuracy_score(y_true, y_pred)
mae  = mean_absolute_error(y_true, y_pred)
hits = (y_true == y_pred).sum()

print(f"\n🎯 Model          : {model_name}")
print(f"📊 Accuracy       : {acc:.1%}  ({hits}/{len(y_true)} correct)")
print(f"📊 MAE (in stars) : {mae:.2f}")

# -----------------------------------------------------------
# 3) Classification report
# -----------------------------------------------------------
labels       = [1, 2, 3, 4, 5]
target_names = [f"{i}★" for i in labels]
print("\n📋 CLASSIFICATION REPORT")
print(
    classification_report(
        y_true, y_pred,
        labels=labels,
        target_names=target_names,
        digits=3,
        zero_division=0
    )
)

# -----------------------------------------------------------
# 4) Confusion matrix
# -----------------------------------------------------------
print("\n🔍 CONFUSION MATRIX")
cm = confusion_matrix(y_true, y_pred, labels=labels)

# header row
print("True\\Pred", *[f"{lbl}★" for lbl in labels], sep="  ")
for i, true_lbl in enumerate(labels):
    row_vals = "  ".join(f"{cm[i, j]:2d}" for j in range(len(labels)))
    print(f"   {true_lbl}★   {row_vals}")

diag    = np.trace(cm)
tot     = cm.sum()
print(f"\n   Correct predictions : {diag}/{tot} ({diag/tot:.1%})")

# most frequent confusions (off-diagonal)
off_diag = [
    (cm[i, j], labels[i], labels[j])
    for i in range(len(labels)) for j in range(len(labels))
    if i != j and cm[i, j] > 0
]
if off_diag:
    off_diag.sort(reverse=True)
    print("   Top confusions:")
    for count, t, p in off_diag[:3]:
        print(f"     {t}★ → {p}★ : {count}×")

# -----------------------------------------------------------
# 5) Per-class hit rate
# -----------------------------------------------------------
print("\n🎭 PER-CLASS ACCURACY")
for lbl in labels:
    mask = y_true == lbl
    if mask.any():
        class_acc = (y_pred[mask] == lbl).mean()
        print(f"   {lbl}★ : {class_acc:.1%} ({mask.sum()} samples)")
    else:
        print(f"   {lbl}★ : no samples in test set")

print("=" * 60)




📈  PERFORMANCE ANALYSIS
🔍 Debug:
   y_true unique : [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5)]
   y_pred unique : [np.int64(3)]

🎯 Model          : Stacking
📊 Accuracy       : 20.0%  (3/15 correct)
📊 MAE (in stars) : 1.20

📋 CLASSIFICATION REPORT
              precision    recall  f1-score   support

          1★      0.000     0.000     0.000         3
          2★      0.000     0.000     0.000         3
          3★      0.200     1.000     0.333         3
          4★      0.000     0.000     0.000         3
          5★      0.000     0.000     0.000         3

    accuracy                          0.200        15
   macro avg      0.040     0.200     0.067        15
weighted avg      0.040     0.200     0.067        15


🔍 CONFUSION MATRIX
True\Pred  1★  2★  3★  4★  5★
   1★    0   0   3   0   0
   2★    0   0   3   0   0
   3★    0   0   3   0   0
   4★    0   0   3   0   0
   5★    0   0   3   0   0

   Correct predictions : 3/15 (20.0%)
   Top confusion

In [19]:
# =============================================================================
# 7) PER-REVIEW DIAGNOSTICS
# =============================================================================
import textwrap

print("\n🔍  DETAILED PREDICTION REVIEW")
print("=" * 80)

for i in range(len(y_true)):
    true_r  = int(y_true[i])
    pred_r  = int(y_pred[i])
    correct = (true_r == pred_r)
    status  = "✅" if correct else "❌"

    print("\n" + "=" * 80)
    print(f"Sample {i+1:2d} | True: {true_r}★ → Pred: {pred_r}★ {status}")

    # -----------------------------------------------------------
    # Confidence bar (if we have probabilities)
    # -----------------------------------------------------------
    if confidence is not None:
        conf = float(confidence[i])
        bar  = "█" * int(conf * 20)
        print(f"Confidence: {conf:.1%} {bar}")

        # print full class probabilities
        if hasattr(clf, "predict_proba"):
            probs = y_proba[i]
            print("All classes:", end=" ")
            for idx, p in enumerate(probs):
                star = idx + 1 if min(y_pred) == 0 else idx + 1  # labels already 1-5 after mapping
                print(f"{star}★:{p:.0%}", end=" ")
            print()

    # -----------------------------------------------------------
    # Show the review text when we loaded from CSV
    # -----------------------------------------------------------
    if sample_texts is not None:
        print("-" * 80)
        snippet = sample_texts[i]
        if len(snippet) > 400:
            snippet = snippet[:400] + " …"
        print(textwrap.fill(snippet, width=76))



🔍  DETAILED PREDICTION REVIEW

Sample  1 | True: 1★ → Pred: 3★ ❌
Confidence: 31.4% ██████
All classes: 1★:25% 2★:26% 3★:31% 4★:16% 5★:2% 

Sample  2 | True: 1★ → Pred: 3★ ❌
Confidence: 35.2% ███████
All classes: 1★:34% 2★:15% 3★:35% 4★:14% 5★:1% 

Sample  3 | True: 1★ → Pred: 3★ ❌
Confidence: 32.0% ██████
All classes: 1★:26% 2★:21% 3★:32% 4★:16% 5★:5% 

Sample  4 | True: 2★ → Pred: 3★ ❌
Confidence: 33.6% ██████
All classes: 1★:19% 2★:21% 3★:34% 4★:17% 5★:10% 

Sample  5 | True: 2★ → Pred: 3★ ❌
Confidence: 35.4% ███████
All classes: 1★:29% 2★:20% 3★:35% 4★:6% 5★:9% 

Sample  6 | True: 2★ → Pred: 3★ ❌
Confidence: 29.9% █████
All classes: 1★:16% 2★:25% 3★:30% 4★:23% 5★:6% 

Sample  7 | True: 3★ → Pred: 3★ ✅
Confidence: 29.4% █████
All classes: 1★:2% 2★:17% 3★:29% 4★:27% 5★:24% 

Sample  8 | True: 3★ → Pred: 3★ ✅
Confidence: 32.4% ██████
All classes: 1★:17% 2★:16% 3★:32% 4★:31% 5★:4% 

Sample  9 | True: 3★ → Pred: 3★ ✅
Confidence: 34.4% ██████
All classes: 1★:26% 2★:15% 3★:34% 4★:14% 5★:1

In [26]:
# =============================================================================
# 8) FEHLERANALYSE
# =============================================================================

print(f"\n❌ FEHLERANALYSE")
print("=" * 50)

# Finde falsche Vorhersagen
wrong_indices = np.where(y_true != y_pred_orig)[0]
wrong_count = len(wrong_indices)

if wrong_count == 0:
    print("🎉 Alle Vorhersagen waren korrekt!")
else:
    print(f"Falsche Vorhersagen: {wrong_count}/{len(y_true)}")
    
    # Analysiere Fehler-Muster
    error_patterns = {}
    for idx in wrong_indices:
        true_r = y_true[idx]
        pred_r = y_pred_orig[idx]
        pattern = f"{true_r}→{pred_r}"
        error_patterns[pattern] = error_patterns.get(pattern, 0) + 1
    
    print("\nFehler-Muster:")
    for pattern, count in sorted(error_patterns.items()):
        true_r, pred_r = pattern.split('→')
        diff = abs(int(true_r) - int(pred_r))
        print(f"   {pattern}: {count}x (Diff: {diff} Sterne)")
    
    # Durchschnittliche Abweichung bei Fehlern
    avg_error = np.mean([abs(y_true[i] - y_pred_orig[i]) for i in wrong_indices])
    print(f"\nØ Abweichung bei Fehlern: {avg_error:.2f} Sterne")


❌ FEHLERANALYSE
Falsche Vorhersagen: 12/15

Fehler-Muster:
   1→4: 3x (Diff: 3 Sterne)
   2→4: 3x (Diff: 2 Sterne)
   3→4: 3x (Diff: 1 Sterne)
   4→6: 1x (Diff: 2 Sterne)
   5→4: 2x (Diff: 1 Sterne)

Ø Abweichung bei Fehlern: 1.83 Sterne


In [27]:
# =============================================================================
# 9) CONFIDENCE-ANALYSE
# =============================================================================

if confidence is not None:
    print(f"\n🎯 CONFIDENCE-ANALYSE")
    print("=" * 50)
    
    avg_conf = confidence.mean()
    min_conf = confidence.min()
    max_conf = confidence.max()
    
    print(f"Durchschnittliche Confidence: {avg_conf:.1%}")
    print(f"Niedrigste Confidence: {min_conf:.1%}")
    print(f"Höchste Confidence: {max_conf:.1%}")
    
    # Confidence vs. Korrektheit
    correct_mask = (y_true == y_pred_orig)
    if np.any(correct_mask):
        avg_conf_correct = confidence[correct_mask].mean()
        print(f"Confidence bei korrekten Vorhersagen: {avg_conf_correct:.1%}")
    
    if np.any(~correct_mask):
        avg_conf_wrong = confidence[~correct_mask].mean()
        print(f"Confidence bei falschen Vorhersagen: {avg_conf_wrong:.1%}")
    
    # Niedrigste Confidence Fälle
    low_conf_indices = np.argsort(confidence)[:3]
    print(f"\nNiedrigste Confidence Fälle:")
    for idx in low_conf_indices:
        true_r, pred_r = y_true[idx], y_pred_orig[idx]
        conf = confidence[idx]
        status = "✅" if true_r == pred_r else "❌"
        print(f"   Sample {idx+1}: {true_r}⭐→{pred_r}⭐ {status} ({conf:.1%})")


🎯 CONFIDENCE-ANALYSE
Durchschnittliche Confidence: 31.4%
Niedrigste Confidence: 29.3%
Höchste Confidence: 33.4%
Confidence bei korrekten Vorhersagen: 31.4%
Confidence bei falschen Vorhersagen: 31.4%

Niedrigste Confidence Fälle:
   Sample 4: 2⭐→4⭐ ❌ (29.3%)
   Sample 7: 3⭐→4⭐ ❌ (30.5%)
   Sample 15: 5⭐→4⭐ ❌ (30.5%)


In [ ]:
# =============================================================================
# 12) INTERAKTIVER TEST
# =============================================================================

def test_custom_text(text_input):
    """Teste einen einzelnen Text - KORRIGIERTE VERSION"""
    try:
        # Feature-Extraktion basierend auf Modell-Typ
        model_features = mdl_dict.get("feature_combination", "combined")
        
        if model_features == "tfidf_only":
            X_custom = vectorizer.transform([text_input])
        elif model_features == "numerical_only":
            print("⚠️ Custom Text Test mit 'numerical_only' nicht möglich (keine Textfeatures)")
            return None
        else:  # combined - hier war der Hauptfehler
            X_tfidf = vectorizer.transform([text_input])
            
            X_custom = X_tfidf
        
        pred_enc = clf.predict(X_custom)[0]
        pred_orig = label_encoder.inverse_transform([pred_enc])[0]
        
        result = {
            'prediction': pred_orig,
            'confidence': None
        }
        
        if hasattr(clf, "predict_proba"):
            print("🔮 Vorhersage mit Wahrscheinlichkeiten:")
            proba = clf.predict_proba(X_custom)[0]
            result['confidence'] = proba.max()
            result['all_probabilities'] = {
                label_encoder.inverse_transform([i])[0]: prob 
                for i, prob in enumerate(proba)
            }
        
        return result
        
    except Exception as e:
        print(f"❌ Fehler beim Custom Test: {e}")
        return None

# Beispiel für Custom Test (Automatisch)
print(f"\n🎮 BEISPIEL: CUSTOM TEXT TEST")
print("=" * 50)

test_texts = [
    "This product is amazing! Best purchase ever!",
    "Terrible quality, broke immediately. Don't buy!",
    "It's okay, nothing special but does the job."
]

for i, test_text in enumerate(test_texts):
    result = test_custom_text(test_text)
    if result:
        print(f"\nTest {i+1}: {result['prediction']}⭐", end="")
        if result['confidence']:
            print(f" (Confidence: {result['confidence']:.1%})")
        else:
            print()
        print(f"Text: {test_text}")


🎮 BEISPIEL: CUSTOM TEXT TEST
testtst <Compressed Sparse Row sparse matrix of dtype 'float64'
	with 5 stored elements and shape (1, 5010)>
  Coords	Values
  (0, 98)	0.4261538241554763
  (0, 306)	0.40489382633340476
  (0, 3164)	0.24615886958176456
  (0, 3165)	0.702660449165133
  (0, 3243)	0.31643010548941036
🔮 Vorhersage mit Wahrscheinlichkeiten:

Test 1: 2⭐ (Confidence: 41.8%)
Text: This product is amazing! Best purchase ever!
testtst <Compressed Sparse Row sparse matrix of dtype 'float64'
	with 6 stored elements and shape (1, 5010)>
  Coords	Values
  (0, 392)	0.49506702537777625
  (0, 422)	0.2524125041938558
  (0, 1887)	0.3777895866716374
  (0, 3265)	0.2086605463212705
  (0, 4418)	0.38568962989905486
  (0, 4423)	0.5968047269500295
🔮 Vorhersage mit Wahrscheinlichkeiten:

Test 2: 3⭐ (Confidence: 42.7%)
Text: Terrible quality, broke immediately. Don't buy!
testtst <Compressed Sparse Row sparse matrix of dtype 'float64'
	with 3 stored elements and shape (1, 5010)>
  Coords	Values
  (0, 21

In [71]:
# =============================================================================
# 13) INTERAKTIVER EINGABE-TEST 
# =============================================================================

def interactive_prediction():
    """Interaktive Vorhersage für eigene Texte - NEUE VERSION"""
    print("\n" + "="*60)
    print("🎮 INTERACTIVE MODE - Teste eigene Reviews!")
    print("(Gib 'quit' ein zum Beenden)")
    print("="*60)
    
    counter = 1
    while True:
        try:
            user_text = input(f"\n[{counter}] 📝 Gib einen Review-Text ein: ").strip()
            
            if user_text.lower() in ['quit', 'exit', 'q', '']:
                print("👋 Interaktiver Test beendet!")
                break
                
            if not user_text:
                print("❌ Bitte gib einen Text ein!")
                continue
            
            # Vorhersage mit korrigierter Funktion
            result = test_custom_text(user_text)
            
            if result:
                print(f"\n🔮 ERGEBNIS FÜR EINGABE #{counter}:")
                print("-" * 50)
                print(f"Eingabe: {user_text}")
                print(f"Vorhersage: {result['prediction']}⭐")
                
                if result['confidence']:
                    conf_bar = "█" * int(result['confidence'] * 20) + "░" * (20 - int(result['confidence'] * 20))
                    print(f"Confidence: {result['confidence']:.1%} [{conf_bar}]")
                
                if 'all_probabilities' in result:
                    print(f"\n📊 WAHRSCHEINLICHKEITSVERTEILUNG:")
                    print("-" * 50)
                    print(f"{'Rating':<8} {'Wahrscheinlichkeit':<15} {'Visualisierung'}")
                    print("-" * 50)
                    
                    for rating in sorted(result['all_probabilities'].keys()):
                        prob = result['all_probabilities'][rating]
                        bar = "█" * int(prob * 25)
                        print(f"{rating}⭐{'':<6} {prob:.1%}{'':<12} {bar}")
                
                print("-" * 50)
            else:
                print("❌ Fehler bei der Vorhersage")
            
            counter += 1
            
        except KeyboardInterrupt:
            print("\n👋 Abgebrochen durch Benutzer!")
            break
        except Exception as e:
            print(f"❌ Fehler: {e}")

# Starte den interaktiven Modus
interactive_prediction()


🎮 INTERACTIVE MODE - Teste eigene Reviews!
(Gib 'quit' ein zum Beenden)

🔮 ERGEBNIS FÜR EINGABE #1:
--------------------------------------------------
Eingabe: geniales product
Vorhersage: 2⭐
Confidence: 45.9% [█████████░░░░░░░░░░░]

📊 WAHRSCHEINLICHKEITSVERTEILUNG:
--------------------------------------------------
Rating   Wahrscheinlichkeit Visualisierung
--------------------------------------------------
1⭐       14.2%             ███
2⭐       45.9%             ███████████
3⭐       32.5%             ████████
4⭐       5.7%             █
5⭐       1.7%             
--------------------------------------------------

🔮 ERGEBNIS FÜR EINGABE #2:
--------------------------------------------------
Eingabe: ich mag das product sehr und würde es wieder kaufen
Vorhersage: 2⭐
Confidence: 45.9% [█████████░░░░░░░░░░░]

📊 WAHRSCHEINLICHKEITSVERTEILUNG:
--------------------------------------------------
Rating   Wahrscheinlichkeit Visualisierung
--------------------------------------------------


In [72]:
# Verbesserte Test-Beispiele für verschiedene Sterne-Kategorien

# 5 STERNE - Sehr positive Reviews
five_star_examples = [
    "Absolutely amazing product! Exceeded all my expectations. Fast shipping, perfect quality, exactly as described. Will definitely order again from this seller. Highly recommend to everyone!",
    "Outstanding quality for the price! The item arrived quickly and was packaged very well. Works perfectly and looks even better than in the photos. Customer service was excellent too.",
    "Perfect purchase! This product is fantastic, great value for money. The seller was responsive and helpful. Shipping was super fast. Could not be happier with this buy!",
    "Exceptional item! Top quality materials, exactly what I needed. The seller communicated well and shipped immediately. This exceeded my expectations in every way possible."
]

# 4 STERNE - Positive Reviews mit kleinen Einschränkungen
four_star_examples = [
    "Great product overall! Good quality and works as expected. Shipping took a bit longer than anticipated but worth the wait. Would recommend this to others.",
    "Very satisfied with this purchase. The item is well made and functional. Only minor issue was the packaging could have been better, but the product itself is excellent.",
    "Good quality product at a reasonable price. Delivery was on time and the item matches the description. Small scratches on arrival but nothing major.",
    "Happy with this buy! The product works well and looks good. Installation was straightforward. Only wish it came with better instructions."
]

# 3 STERNE - Neutrale/gemischte Reviews
three_star_examples = [
    "Average product. It does what it's supposed to do but nothing special. Quality is okay for the price. Shipping was standard.",
    "Mixed feelings about this purchase. Some aspects are good, others could be improved. It works but feels a bit cheap. Decent value overall.",
    "It's an okay product. Not bad but not great either. Quality is acceptable and it serves its purpose. Would consider other options next time.",
    "Mediocre item. Gets the job done but I expected better quality. Shipping was fine. It's functional but not impressive."
]

# 2 STERNE - Negative Reviews mit einigen positiven Aspekten
two_star_examples = [
    "Disappointed with this purchase. The quality is poor and it feels flimsy. Only positive is that shipping was fast. Would not buy again.",
    "Not what I expected. The item looks different from the photos and feels cheap. It works but barely. Customer service was unhelpful.",
    "Poor quality product. Had issues right after opening. The material feels very cheap. Only reason for 2 stars is that it arrived on time.",
    "Unsatisfied with this buy. The product is smaller than expected and build quality is questionable. Shipping was okay but that's about it."
]

# 1 STERN - Sehr negative Reviews
one_star_examples = [
    "Terrible product! Completely useless and broke immediately after opening. Worst purchase ever. Poor quality, misleading description. Avoid this seller!",
    "Absolute waste of money! The item is nothing like described, extremely poor quality. Took forever to arrive and was damaged. Complete scam!",
    "Horrible experience! Product is defective and unusable. Seller refuses to help or refund. Cheap plastic junk that falls apart instantly.",
    "Do not buy this! Completely broken on arrival, terrible packaging. Item looks nothing like the photos. Seller is unresponsive. Total disaster!"
]

# Alle Beispiele kombinieren für systematischen Test
all_test_examples = {
    5: five_star_examples,
    4: four_star_examples, 
    3: three_star_examples,
    2: two_star_examples,
    1: one_star_examples
}

print("🧪 SYSTEMATISCHER TEST MIT VERBESSERTEN BEISPIELEN")
print("="*70)

total_correct = 0
total_tests = 0

for expected_stars, examples in all_test_examples.items():
    print(f"\n⭐ ERWARTETE {expected_stars} STERNE:")
    print("-" * 50)
    
    for i, text in enumerate(examples, 1):
        result = test_custom_text(text)
        if result:
            predicted = result['prediction']
            confidence = result.get('confidence', 0)
            is_correct = predicted == expected_stars
            status = "✅" if is_correct else "❌"
            
            total_tests += 1
            if is_correct:
                total_correct += 1
            
            print(f"{status} Beispiel {i}: {predicted}⭐ (Conf: {confidence:.1%})")
            print(f"   Text: {text[:80]}{'...' if len(text) > 80 else ''}")
            
            # Zeige Wahrscheinlichkeitsverteilung bei falschen Vorhersagen
            if not is_correct and 'all_probabilities' in result:
                print("   Verteilung:", end=" ")
                for rating in sorted(result['all_probabilities'].keys()):
                    prob = result['all_probabilities'][rating]
                    print(f"{rating}⭐:{prob:.0%}", end=" ")
                print()
        print()

print(f"\n📊 GESAMTERGEBNIS:")
print(f"Korrekte Vorhersagen: {total_correct}/{total_tests} ({total_correct/total_tests*100:.1f}%)")

# Zusätzliche Analyse: Welche Wörter/Phrasen führen zu welchen Bewertungen?
print(f"\n🔍 WORT-ANALYSE:")
print("Teste einzelne charakteristische Wörter/Phrasen...")

word_tests = {
    "amazing": 5,
    "excellent": 5, 
    "perfect": 5,
    "outstanding": 5,
    "terrible": 1,
    "horrible": 1,
    "waste of money": 1,
    "broke immediately": 1,
    "good quality": 4,
    "works well": 4,
    "okay": 3,
    "average": 3,
    "poor quality": 2,
    "disappointed": 2
}

print("\nWort/Phrase → Vorhersage:")
for phrase, expected in word_tests.items():
    result = test_custom_text(phrase)
    if result:
        predicted = result['prediction']
        status = "✅" if predicted == expected else "❌"
        print(f"{status} '{phrase}' → {predicted}⭐ (erwartet: {expected}⭐)")

🧪 SYSTEMATISCHER TEST MIT VERBESSERTEN BEISPIELEN

⭐ ERWARTETE 5 STERNE:
--------------------------------------------------
❌ Beispiel 1: 2⭐ (Conf: 41.2%)
   Text: Absolutely amazing product! Exceeded all my expectations. Fast shipping, perfect...
   Verteilung: 1⭐:8% 2⭐:41% 3⭐:35% 4⭐:11% 5⭐:4% 

❌ Beispiel 2: 3⭐ (Conf: 42.1%)
   Text: Outstanding quality for the price! The item arrived quickly and was packaged ver...
   Verteilung: 1⭐:6% 2⭐:41% 3⭐:42% 4⭐:9% 5⭐:2% 

❌ Beispiel 3: 2⭐ (Conf: 44.0%)
   Text: Perfect purchase! This product is fantastic, great value for money. The seller w...
   Verteilung: 1⭐:12% 2⭐:44% 3⭐:33% 4⭐:8% 5⭐:3% 

❌ Beispiel 4: 3⭐ (Conf: 48.1%)
   Text: Exceptional item! Top quality materials, exactly what I needed. The seller commu...
   Verteilung: 1⭐:3% 2⭐:34% 3⭐:48% 4⭐:12% 5⭐:2% 


⭐ ERWARTETE 4 STERNE:
--------------------------------------------------
❌ Beispiel 1: 2⭐ (Conf: 48.9%)
   Text: Great product overall! Good quality and works as expected. Shipping

In [73]:
# =============================================================================
# MODELL-DIAGNOSE - Was läuft schief?
# =============================================================================

print("🔍 DETAILLIERTE MODELL-DIAGNOSE")
print("="*60)

# 1. Modell-Informationen prüfen
print("1️⃣ MODELL-INFORMATIONEN:")
print("-" * 30)
print(f"Modell-Typ: {type(clf).__name__}")
print(f"Modell-Name: {model_name}")
print(f"Feature-Kombination: {mdl_dict.get('feature_combination', 'unknown')}")

# 2. Label-Encoder prüfen
print(f"\n2️⃣ LABEL-ENCODING:")
print("-" * 30)
print(f"Original Classes: {label_encoder.classes_}")
print("Mapping:")
for i, orig_label in enumerate(label_encoder.classes_):
    print(f"  {orig_label} (original) → {i} (encoded)")

# 3. Vectorizer prüfen
print(f"\n3️⃣ VECTORIZER-INFO:")
print("-" * 30)
print(f"Vectorizer-Typ: {type(vectorizer).__name__}")
print(f"Feature-Count: {len(vectorizer.get_feature_names_out()):,}")
print(f"Max Features: {getattr(vectorizer, 'max_features', 'unlimited')}")

# Test einzelner Wörter im Vokabular
test_words = ['amazing', 'terrible', 'good', 'bad', 'excellent']
vocab = vectorizer.get_feature_names_out()
print(f"\nWörter im Vokabular:")
for word in test_words:
    in_vocab = word in vocab
    print(f"  '{word}': {'✅' if in_vocab else '❌'}")

# 4. Feature-Matrix für Test-Wörter analysieren
print(f"\n4️⃣ FEATURE-MATRIX ANALYSE:")
print("-" * 30)

test_phrases = ['amazing', 'terrible', 'good quality', 'poor quality']
for phrase in test_phrases:
    X_test = vectorizer.transform([phrase])
    print(f"\nPhrase: '{phrase}'")
    print(f"  Sparse Matrix Shape: {X_test.shape}")
    print(f"  Non-zero Features: {X_test.nnz}")
    
    # Zeige die aktivierten Features
    if X_test.nnz > 0:
        feature_indices = X_test.nonzero()[1]
        feature_names = vocab[feature_indices]
        feature_values = X_test.data
        print(f"  Aktivierte Features: {list(zip(feature_names, feature_values))[:5]}")

# 5. Modell-Vorhersage-Verteilung
print(f"\n5️⃣ VORHERSAGE-VERTEILUNG:")
print("-" * 30)

# Teste viele zufällige Texte
random_texts = [
    "good", "bad", "excellent", "terrible", "okay", 
    "amazing product", "poor quality", "works well", "broken",
    "love it", "hate it", "perfect", "awful", "decent"
]

predictions = []
for text in random_texts:
    X = vectorizer.transform([text])
    pred_encoded = clf.predict(X)[0]
    pred_original = label_encoder.inverse_transform([pred_encoded])[0]
    predictions.append(pred_original)

from collections import Counter
pred_counts = Counter(predictions)
print("Vorhersage-Verteilung für Test-Texte:")
for rating in sorted(pred_counts.keys()):
    count = pred_counts[rating]
    percentage = count / len(predictions) * 100
    bar = "█" * int(percentage / 5)
    print(f"  {rating}⭐: {count:2d}/{len(predictions)} ({percentage:4.1f}%) {bar}")

# 6. Probability-Analyse
print(f"\n6️⃣ WAHRSCHEINLICHKEITS-ANALYSE:")
print("-" * 30)

if hasattr(clf, 'predict_proba'):
    # Teste extreme Fälle
    extreme_tests = {
        "amazing excellent perfect outstanding": "5⭐ erwartet",
        "terrible horrible awful waste broken": "1⭐ erwartet"
    }
    
    for text, expected in extreme_tests.items():
        X = vectorizer.transform([text])
        probabilities = clf.predict_proba(X)[0]
        pred_encoded = clf.predict(X)[0]
        pred_original = label_encoder.inverse_transform([pred_encoded])[0]
        
        print(f"\nText: '{text}' ({expected})")
        print(f"Vorhersage: {pred_original}⭐")
        print("Wahrscheinlichkeiten:")
        for i, prob in enumerate(probabilities):
            original_rating = label_encoder.inverse_transform([i])[0]
            print(f"  {original_rating}⭐: {prob:.3f}")

# 7. Modell-Parameter prüfen
print(f"\n7️⃣ MODELL-PARAMETER:")
print("-" * 30)
if hasattr(clf, 'get_params'):
    params = clf.get_params()
    important_params = ['C', 'gamma', 'kernel', 'class_weight', 'random_state']
    for param in important_params:
        if param in params:
            print(f"  {param}: {params[param]}")

# 8. Training-Metriken nochmal prüfen
print(f"\n8️⃣ TRAINING-METRIKEN:")
print("-" * 30)
for key, value in mdl_dict.items():
    if key not in ['model', 'model_name']:
        print(f"  {key}: {value}")

# 9. DIAGNOSE-SCHLUSSFOLGERUNGEN
print(f"\n9️⃣ DIAGNOSE-SCHLUSSFOLGERUNGEN:")
print("-" * 30)

issues = []

# Check 1: Alle Vorhersagen in 2-3 Bereich?
unique_preds = set(predictions)
if len(unique_preds) <= 2 and all(p in [2, 3] for p in unique_preds):
    issues.append("❌ Modell vorhersagt nur 2-3 Sterne (bias Problem)")

# Check 2: Sehr schlechte Accuracy auf einfachen Fällen
simple_accuracy = sum(1 for p in predictions if p in [1, 4, 5]) / len(predictions)
if simple_accuracy < 0.2:
    issues.append("❌ Kann extreme Bewertungen (1⭐, 5⭐) nicht erkennen")

# Check 3: Training-Accuracy vs. Test-Performance
train_acc = mdl_dict.get('train_accuracy', 0)
if train_acc > 0.8:
    issues.append("❌ Mögliches Overfitting (gute Training-Acc aber schlechte Real-World Performance)")

# Check 4: Label-Encoding Problem?
if not all(isinstance(c, int) and 1 <= c <= 5 for c in label_encoder.classes_):
    issues.append("❌ Label-Encoder Problem - falsche Klassen")

print("\n🚨 GEFUNDENE PROBLEME:")
if issues:
    for issue in issues:
        print(f"  {issue}")
else:
    print("  ✅ Keine offensichtlichen Probleme gefunden")

print(f"\n💡 LÖSUNGSVORSCHLÄGE:")
print("  1. Modell komplett neu trainieren")
print("  2. Andere Algorithmen testen (Random Forest, XGBoost)")
print("  3. Hyperparameter-Tuning verbessern")
print("  4. Datenqualität prüfen (sind die Labels korrekt?)")
print("  5. Feature-Engineering überdenken")
print("  6. Class Imbalance behandeln (SMOTE, Class Weights)")

🔍 DETAILLIERTE MODELL-DIAGNOSE
1️⃣ MODELL-INFORMATIONEN:
------------------------------
Modell-Typ: StackingClassifier
Modell-Name: Optimized Stacking
Feature-Kombination: combined

2️⃣ LABEL-ENCODING:
------------------------------
Original Classes: [1 2 3 4 5]
Mapping:
  1 (original) → 0 (encoded)
  2 (original) → 1 (encoded)
  3 (original) → 2 (encoded)
  4 (original) → 3 (encoded)
  5 (original) → 4 (encoded)

3️⃣ VECTORIZER-INFO:
------------------------------
Vectorizer-Typ: TfidfVectorizer
Feature-Count: 5,010
Max Features: 5010

Wörter im Vokabular:
  'amazing': ✅
  'terrible': ✅
  'good': ✅
  'bad': ✅
  'excellent': ✅

4️⃣ FEATURE-MATRIX ANALYSE:
------------------------------

Phrase: 'amazing'
  Sparse Matrix Shape: (1, 5010)
  Non-zero Features: 1
  Aktivierte Features: [('amazing', np.float64(1.0))]

Phrase: 'terrible'
  Sparse Matrix Shape: (1, 5010)
  Non-zero Features: 1
  Aktivierte Features: [('terrible', np.float64(1.0))]

Phrase: 'good quality'
  Sparse Matrix Shape

In [67]:
# =============================================================================
# 13) ZUSAMMENFASSUNG
# =============================================================================

print(f"\n{'='*80}")
print("📋 ZUSAMMENFASSUNG")
print("="*80)

print(f"🎯 Modell: {model_name}")
print(f"🔧 Feature-Kombination: {mdl_dict.get('feature_combination', 'unknown')}")
print(f"📊 Test Samples: {len(y_true)}")
print(f"📊 Accuracy: {accuracy:.1%}")
print(f"📊 MAE: {mae:.2f} Sterne")
print(f"✅ Richtige Vorhersagen: {correct_count}")
print(f"❌ Falsche Vorhersagen: {len(y_true) - correct_count}")

if confidence is not None:
    print(f"🎯 Durchschnittliche Confidence: {confidence.mean():.1%}")

print(f"\n📂 Datenquelle: {'Train/Test Splits' if data_source == 'splits' else 'Preprocessed CSV'}")
print(f"🔧 Features: {X_sample.shape[1]:,}")
print(f"🏷️ Labels: {sorted(label_encoder.classes_)}")
if numerical_features:
    print(f"📊 Numerical Features: {numerical_features}")

# Training-Informationen - PRIMÄR F1 MACRO
macro_f1 = mdl_dict.get('macro_f1', 'N/A')
train_acc = mdl_dict.get('test_accuracy', 'N/A')
best_params = mdl_dict.get('best_params', {})

if macro_f1 != 'N/A':
    print(f"🎯 Training Macro F1-Score: {macro_f1:.3f}")
if train_acc != 'N/A':
    print(f"🎯 Training Test Accuracy: {train_acc:.3f}")
if best_params:
    print(f"⚙️ Best Params: {best_params}")

print(f"\n🚀 INFERENCE DEMO ABGESCHLOSSEN!")

# Optional: Ergebnisse speichern
# results_df = pd.DataFrame({
#     'true_rating': y_true,
#     'predicted_rating': y_pred_orig,
#     'correct': y_true == y_pred_orig,
#     'confidence': confidence if confidence is not None else [None] * len(y_true)
# })
# results_df.to_csv("inference_results.csv", index=False)
# print(f"\n💾 Ergebnisse gespeichert als 'inference_results.csv'")


📋 ZUSAMMENFASSUNG
🎯 Modell: Optimized Stacking
🔧 Feature-Kombination: combined
📊 Test Samples: 15
📊 Accuracy: 20.0%
📊 MAE: 1.00 Sterne
✅ Richtige Vorhersagen: 3
❌ Falsche Vorhersagen: 12
🎯 Durchschnittliche Confidence: 44.8%

📂 Datenquelle: Train/Test Splits
🔧 Features: 5,010
🏷️ Labels: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5)]
📊 Numerical Features: ['word_count', 'char_count', 'sentence_count', 'avg_word_length', 'exclamation_count', 'question_count', 'capital_ratio', 'sentiment_compound', 'sentiment_pos', 'sentiment_neg']
🎯 Training Macro F1-Score: 0.443
🎯 Training Test Accuracy: 0.604

🚀 INFERENCE DEMO ABGESCHLOSSEN!
